In [ ]:
from aiida import load_profile, orm
load_profile();

from pathlib import Path
from aiida.common.datastructures import StashMode
import aiidalab_widgets_base as awb

import ipywidgets as widgets
from IPython.display import display, clear_output
from widgets import (command,
                     releases,
                     input_phy,
                     basic,
                     misc,
                     presettings,
                     code_setup
                     )
from utils import utils
from settings import *
path_config = Path.cwd()/'config/'

In [ ]:
out = widgets.Output()

# FLEXPART-COSMO/IFS Simulation 


Checklist:<br>
<ol>
    <li>The <b>ssh</b> conection is valid.</li>
    <li><b>Codes</b> are selected (computers and codes have been previously configured and tested).</li>
    <li><b>Cache</b> is enabled.</li>
</ol>

[Guide](./info.ipynb)

In [ ]:
list_files = ['COMMAND',
            'INPUT_PHY',
            'RELEASE',
            'Queueing settings'
            ]
co = command.CommentFile()
in_phy = input_phy.InputPhy()
rel = releases.Releases()
basic_inputs = basic.Basic()
misc = misc.Misc()
code_setup = code_setup.CodeSetup()
pre = presettings.Presettings(co,
                              in_phy,
                              rel,
                              basic_inputs)

tab = widgets.Tab(children=[co, 
                            in_phy,
                            rel,
                            misc
                            ])
                            
for i,file_name in enumerate(list_files):
    tab.set_title(i,file_name)

main_tab = widgets.Tab(children = [basic_inputs,
                                   tab,
                                   code_setup
                                   ])
main_tab.set_title(0,'Basic')
main_tab.set_title(1,'Advanced')
main_tab.set_title(2,'Code setup')
main_tab

In [ ]:
def prepare_workflow():  

    list_locations = [i for i in basic_inputs.locations.fill()]

    with out:
        clear_output()

    if not basic_inputs.simulation_dates.value:
        can_submit, msg = False, "Please select dates."
    elif not basic_inputs.model.value or not basic_inputs.model_offline.value:
        can_submit, msg = False, "Please select models."
    elif not basic_inputs.outgrid.selected_outgrid():
        can_submit, msg = False, "Please select an outgrid."
    elif basic_inputs.model_offline.value != 'None' and basic_inputs.offline_integration_time.value == 0:
        can_submit, msg = False, "Model offline is not empty but offline_integration_time is zero."
    elif basic_inputs.model_offline.value is 'None' and basic_inputs.offline_integration_time.value != 0:
        can_submit, msg = False, "Model offline is None but offline_integration_time is not zero."
    elif not list_locations:
        can_submit, msg = False, "Please select locations"
    else:
        can_submit = True

    if not can_submit:
        with out:
            print(msg)
            return
    

    prepend_text_ = f'#SBATCH --partition={misc.partition.value}\n'+\
                    f'#SBATCH --account={misc.account.value}\n'+\
                    misc.prepend_text.value
    
    computer = orm.load_computer(misc.computer.value)
    simulation_dates = utils.simulation_dates_parser([basic_inputs.simulation_dates.value])
    model = basic_inputs.model.value
    model_offline = basic_inputs.model_offline.value
    username = computer.get_configuration()['username']
    outgrid_main = basic_inputs.outgrid.selected_outgrid()
    integration_time = basic_inputs.integration_time.value
    integration_time_offline = basic_inputs.offline_integration_time.value
    users_address = f'/users/{username}/resources/flexpart/'
    scratch_address = f'/scratch/snx3000/{username}/FLEXPART_input/'
    
    # Links to the remote files/folders.
    glc = orm.RemoteData(remote_path = users_address+'GLC2000',
                         computer=computer)
    glc_ifs = orm.RemoteData(remote_path = users_address+'IGBP_int1.dat',
                         computer=computer)
    species = orm.RemoteData(
        remote_path = users_address+'SPECIES',
        computer=computer)
    surfdata = orm.RemoteData(
        remote_path = users_address+'surfdata.t',
        computer=computer)
    surfdepo = orm.RemoteData(
        remote_path = users_address+'surfdepo.t',
        computer=computer)
    
    #builder starts
    builder = WORKFLOW.get_builder()

    #codes
    builder.fcosmo_code = orm.load_code(misc.f_cosmo_code.value)
    builder.fifs_code = orm.load_code(misc.f_ifs_code.value)
    builder.check_meteo_ifs_code = orm.load_code(misc.ifs_m_code.value)
    builder.check_meteo_cosmo_code = orm.load_code(misc.cosmo_m_code.value)
    builder.post_processing_code = orm.load_code(misc.f_post_code.value)

    #prepend text
    builder.flexpartcosmo.metadata.options.custom_scheduler_commands = prepend_text_
    builder.flexpartifs.metadata.options.custom_scheduler_commands = prepend_text_
    builder.flexpartpost.metadata.options.custom_scheduler_commands = prepend_text_

    #max wall time
    builder.flexpartifs.metadata.options.max_wallclock_seconds=misc.wall_time_ifs.value
    builder.flexpartcosmo.metadata.options.max_wallclock_seconds=misc.wall_time_cosmo.value

    #basic settings
    builder.simulation_dates = simulation_dates
    builder.integration_time = orm.Int(integration_time)
    builder.offline_integration_time = orm.Int(integration_time_offline)

    #meteo realted settings
    model = model.split(',')
    model_offline = model_offline.split(',')
    builder.model = orm.List(model)
    builder.model_offline = orm.List(model_offline)

   
    meteo_path = orm.List([scratch_address+mod for mod in model])
    builder.meteo_path = meteo_path
    builder.meteo_inputs = orm.Dict(
    dict = utils.read_yaml_data(path_config/'meteo_inputs.yaml', names=[
            model[-1],
        ])[model[-1]])
    
    if model_offline[-1] != 'None':
        meteo_path_offline = orm.List([scratch_address+mod for mod in model_offline])
        builder.meteo_path_offline = meteo_path_offline
        builder.meteo_inputs_offline = orm.Dict(
        dict = utils.read_yaml_data(path_config/'meteo_inputs.yaml', names=[
            model_offline[-1],
        ])[model_offline[-1]])

    builder.gribdir = orm.Str(scratch_address)
   
    #model settings
    command_dict = co.fill_dict()
    command_dict.update({'release_chunk':basic_inputs.release_chunk.value})
    
    builder.command = orm.Dict(dict = command_dict)

    dict_ = utils.read_yaml_data(path_config/'locations.yaml',
                           names=list_locations)
    reformated_dict_locations = utils.reformat_locations(dict_, model[-1])
    builder.locations = orm.Dict(dict=reformated_dict_locations)
    builder.input_phy = orm.Dict(dict = in_phy.fill_dict())
    builder.release_settings = orm.Dict(dict = rel.fill_dict())
    
    #other
    builder.outgrid = orm.Dict(
        dict = utils.read_yaml_data(path_config/'outgrid.yaml', names=[
            outgrid_main,
        ]))

    if basic_inputs.outgrid_nest.selected_outgrid_nest()!='None':
        outgrid_nest = basic_inputs.outgrid_nest.selected_outgrid_nest()
        builder.outgrid_nest = orm.Dict(dict = utils.read_yaml_data(
        path_config/'outgrid.yaml', names=[
            outgrid_nest,
        ])) 
        
    builder.species = species
    builder.land_use = {
        'glc': glc,
        'surfdata': surfdata,
        'surfdepo': surfdepo,
    }
    builder.land_use_ifs = {
        'glc': glc_ifs,
        'surfdata': surfdata,
        'surfdepo': surfdepo,
    }
    builder.flexpartcosmo.metadata.options.stash = {
        'source_list': ['aiida.out','header*','partposit_inst', 'grid_time_*.nc'],
        'target_base': f'/store/empa/em05/{username}/aiida_stash',
        'stash_mode': StashMode.COPY.value,
    }
    builder.flexpartifs.metadata.options.stash = {
        'source_list': ['aiida.out','header*','partposit_inst', 'grid_time_*.nc'],
        'target_base': f'/store/empa/em05/{username}/aiida_stash',
        'stash_mode': StashMode.COPY.value,
    }
    builder.flexpartpost.metadata.options.stash = {
        'source_list': ['aiida.out','boundary_sensitivity_*.nc', 'grid_time_*.nc'],
        'target_base': f'/store/empa/em05/{username}/aiida_stash',
        'stash_mode': StashMode.COPY.value,
    }
    
    if pre.save_settings_b.value == True:
        builder.name = pre.name.value

    return builder

## Transport model config case
<ul>
<li>Select a previous setting or create a new one by choosing a name and clicking on the box.</li>
<li>Presettings store all the input information except for the simulation date/s.</li>
<li>A presetting can be deleted at any time, but be aware that this change is permanent. </li>
</ul>

In [ ]:
pre

## Submit 

In [ ]:
btn_submit_ = awb.SubmitButtonWidget(
    WORKFLOW,
    inputs_generator = prepare_workflow,
    disable_after_submit = False,
    append_output = True,
)
btn_submit_.btn_submit.button_style="success"
display(btn_submit_, out)